# Import Packages

In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy_langdetect import LanguageDetector

# Read in Data

In [3]:
talk_df = pd.read_csv('Data/full_data.csv', index_col = 0)
events = pd.read_csv('talk_events.csv', index_col = 0)

In [4]:
talk_df.shape

(4244, 14)

In [5]:
events.shape

(4244, 1)

In [6]:
talk_df = pd.concat([talk_df, events], axis = 1)

In [7]:
talk_df.shape

(4244, 15)

# Check for missing data

In [8]:
talk_df.head()

,date,speaker,title,url,length,summ,tags,views,transcript,date_recorded,upload_date,occupation,bio,comments,event
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so...",599446,So I've been thinking about how to explain thi...,2019-09-24,2020-01-02,"Business executive, ""co-conspirator""",Ipsita Dasgupta drives the consumption of ente...,7,TED@BCG Mumbai
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'...",501290,This chimpanzee stumbles across a windfall of ...,2020-01-02,2020-01-02,NaN,NaN,--,TED-Ed
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a...",461705,"Recently, I've been declaring to anyone who wo...",2019-12-04,2020-01-02,Dangerous woman,Pat Mitchell is a lifelong advocate for women ...,14,TEDWomen 2019
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ...",880662,"When we're young, we're innocently brave, and ...",2019-12-04,2019-12-23,Human rights and disability activist,Cara E. Yar Khan is an international human rig...,17,TEDWomen 2019
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',...",956498,"OK, I have a question for all of us. You ready...",2019-12-04,2019-12-20,Gymnastics coach,Valorie Kondos Field is the retired head coach...,18,TEDWomen 2019


In [9]:
talk_df.transcript.isna().sum()

595

In [10]:
for index in range(0, len(talk_df)):
    if not isinstance(talk_df.transcript[index], str):
        talk_df.transcript[index] = '--'

In [11]:
(talk_df.transcript == '').sum()

0

In [12]:
(talk_df.transcript == '--').sum()

595

In [30]:
no_transcript = talk_df[talk_df.transcript == '--']

for url in no_transcript.url:
    print('https://www.ted.com' + url)

https://www.ted.com/talks/jean_manuel_izaret_a_new_netflix_style_pricing_model_that_could_make_medical_treatments_affordable_for_all
https://www.ted.com/talks/juniper_fitzgerald_why_we_need_to_stop_stigmatizing_mothers_who_do_sex_work
https://www.ted.com/talks/dorsa_amir_how_the_industrial_revolution_changed_childhood
https://www.ted.com/talks/adar_cohen_3_ways_to_lead_tough_unavoidable_conversations
https://www.ted.com/talks/martha_redbone_sleep_sleep_beauty_bright
https://www.ted.com/talks/maria_popova_an_excerpt_from_figuring
https://www.ted.com/talks/morley_follow_the_sound
https://www.ted.com/talks/david_carroll_how_i_sued_cambridge_analytica_over_my_personal_data
https://www.ted.com/talks/chris_fisher_why_we_should_archive_everything_on_the_planet
https://www.ted.com/talks/ella_al_shamahi_why_archaeology_needs_to_transcend_borders
https://www.ted.com/talks/andrew_nemr_the_sounds_and_sights_of_tap_dance
https://www.ted.com/talks/hiromi_ozaki_how_i_bring_myth_and_magic_to_life
http

In [13]:
transcripts = talk_df[talk_df.transcript != '--']['transcript']

In [14]:
len(transcripts)

3649

In [15]:
transcripts

0       So I've been thinking about how to explain thi...
1       This chimpanzee stumbles across a windfall of ...
2       Recently, I've been declaring to anyone who wo...
3       When we're young, we're innocently brave, and ...
4       OK, I have a question for all of us. You ready...
5       I wanted to be a psychologist since I was a te...
6       In the 4th century BCE, a banker’s son threw t...
7       Chris Anderson: Nick Bostrom. So, you have alr...
8       In June of 2017, I volunteered with a group at...
9       So, on April 23 of 2013, the Associated Press ...
10      A mother and her son trek across an endless de...
11      We are all atomically connected. Fundamentally...
12      In her Auntie An-mei’s home, Jing-Mei reluctan...
13      Now, I know it might be easy to think that mic...
14      Aquay Wunne Kesuk. Kelsey Leonard Nooweesuonk....
15      I am an ideas activist. That means I fight for...
16      William Golding was losing his faith in humani...
17      It was

# Check for English Transcripts

In [16]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
text = 'This is an english text.'
doc = nlp(text)
# document level language detection. Think of it like average language of the document!
print(doc._.language)

{'language': 'en', 'score': 0.9999948109300234}


In [19]:
docs = []
index = 0
for transcript in transcripts:
    doc = nlp(transcript)
    docs.append(doc)
    print(index)
    index += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648


In [39]:
# These three are ted talks that are strictly musical or visual in nature, no words
for index in range(0, len(docs)):
    if docs[index]._.language['score'] < 0.9:
        print(index)

1236
2556
3528


In [23]:
for index in range(0, len(docs)):
    if docs[index]._.language['language'] != 'en':
        print(index)

In [16]:
transcripts.reset_index(inplace = True, drop = True)

In [17]:
transcripts[1236]

'(Guitar music starts)(Cheers)(Cheers)(Music ends)'

In [18]:
transcripts[2556]

'(Music)(Applause)(Music)(Applause)(Music)(Applause)(Music)(Applause)'

In [19]:
transcripts[3528]

'(Applause)(Music)(Applause)'

In [20]:
has_transcript = talk_df[talk_df.transcript != '--']

In [21]:
has_transcript.reset_index(inplace = True, drop = True)

In [22]:
has_transcript.iloc[1236]

date                                                      Feb 2017
speaker                                         Rodrigo y Gabriela
title                  An electrifying acoustic guitar performance
url              /talks/rodrigo_y_gabriela_an_electrifying_acou...
length                                                        4:17
summ             Guitar duo Rodrigo y Gabriela combine furiousl...
tags             ['music', 'live music', 'performance', 'guitar...
views                                                      2681680
transcript       (Guitar music starts)(Cheers)(Cheers)(Music ends)
date_recorded                                           2015-03-16
upload_date                                             2017-02-14
occupation                                              Guitar duo
bio              Rodrigo y Gabriela fuse metal, jazz and world ...
comments                                                        39
event                                                      TED

In [23]:
has_transcript.iloc[2556]

date                                                      Jun 2012
speaker                                            Quixotic Fusion
title                                           Dancing with light
url                      /talks/quixotic_fusion_dancing_with_light
length                                                       12:22
summ             Quixotic Fusion is an ensemble of artists that...
tags             ['culture', 'dance', 'entertainment', 'technol...
views                                                      1589575
transcript       (Music)(Applause)(Music)(Applause)(Music)(Appl...
date_recorded                                           2012-02-28
upload_date                                             2012-06-01
occupation                                    Performance ensemble
bio              Anthony Magliano and Mica Thomas are the found...
comments                                                       109
event                                                      TED

In [24]:
has_transcript.iloc[3528]

date                                                      Oct 2007
speaker                                              Kenichi Ebina
title                                               My magic moves
url                            /talks/kenichi_ebina_my_magic_moves
length                                                        3:32
summ             Kenichi Ebina moves his body in a manner that ...
tags             ['culture', 'dance', 'entertainment', 'perform...
views                                                      1921963
transcript                             (Applause)(Music)(Applause)
date_recorded                                           2007-03-03
upload_date                                             2007-10-03
occupation                                                  Dancer
bio              2013 "America's Got Talent" winner Kenichi Ebi...
comments                                                       103
event                                                      TED

### Drop Ted Talks that don't have transcript

In [25]:
has_transcript.drop([1236, 2556, 3528], inplace = True)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [26]:
has_transcript.shape

(3646, 15)

# Convert duration to seconds

In [27]:
has_transcript.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3646 entries, 0 to 3648
Data columns (total 15 columns):
date             3646 non-null object
speaker          3646 non-null object
title            3646 non-null object
url              3646 non-null object
length           3646 non-null object
summ             3646 non-null object
tags             3646 non-null object
views            3646 non-null object
transcript       3646 non-null object
date_recorded    3646 non-null object
upload_date      3646 non-null object
occupation       3223 non-null object
bio              3235 non-null object
comments         3646 non-null object
event            3646 non-null object
dtypes: object(15)
memory usage: 455.8+ KB


In [28]:
has_transcript.head()

,date,speaker,title,url,length,summ,tags,views,transcript,date_recorded,upload_date,occupation,bio,comments,event
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so...",599446,So I've been thinking about how to explain thi...,2019-09-24,2020-01-02,"Business executive, ""co-conspirator""",Ipsita Dasgupta drives the consumption of ente...,7,TED@BCG Mumbai
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'...",501290,This chimpanzee stumbles across a windfall of ...,2020-01-02,2020-01-02,NaN,NaN,--,TED-Ed
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a...",461705,"Recently, I've been declaring to anyone who wo...",2019-12-04,2020-01-02,Dangerous woman,Pat Mitchell is a lifelong advocate for women ...,14,TEDWomen 2019
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ...",880662,"When we're young, we're innocently brave, and ...",2019-12-04,2019-12-23,Human rights and disability activist,Cara E. Yar Khan is an international human rig...,17,TEDWomen 2019
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',...",956498,"OK, I have a question for all of us. You ready...",2019-12-04,2019-12-20,Gymnastics coach,Valorie Kondos Field is the retired head coach...,18,TEDWomen 2019


In [29]:
has_transcript.length.isna().sum()

0

In [30]:
has_transcript['duration'] = [int(length.split(':')[0]) * 60 + int(length.split(':')[1]) for length in has_transcript.length]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
has_transcript.head()

,date,speaker,title,url,length,summ,tags,views,transcript,date_recorded,upload_date,occupation,bio,comments,event,duration
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so...",599446,So I've been thinking about how to explain thi...,2019-09-24,2020-01-02,"Business executive, ""co-conspirator""",Ipsita Dasgupta drives the consumption of ente...,7,TED@BCG Mumbai,663
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'...",501290,This chimpanzee stumbles across a windfall of ...,2020-01-02,2020-01-02,NaN,NaN,--,TED-Ed,296
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a...",461705,"Recently, I've been declaring to anyone who wo...",2019-12-04,2020-01-02,Dangerous woman,Pat Mitchell is a lifelong advocate for women ...,14,TEDWomen 2019,1034
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ...",880662,"When we're young, we're innocently brave, and ...",2019-12-04,2019-12-23,Human rights and disability activist,Cara E. Yar Khan is an international human rig...,17,TEDWomen 2019,595
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',...",956498,"OK, I have a question for all of us. You ready...",2019-12-04,2019-12-20,Gymnastics coach,Valorie Kondos Field is the retired head coach...,18,TEDWomen 2019,949


In [42]:
has_transcript.to_csv('has_transcript.csv')

# Replace -- with np.nan, convert columns to appropriate types

In [32]:
has_transcript.views = has_transcript.views.replace('--', np.nan)
has_transcript.views = has_transcript.views.astype(float)

has_transcript.comments = has_transcript.comments.replace('--', np.nan)
has_transcript.comments = has_transcript.comments.astype(float)

has_transcript.date_recorded = has_transcript.date_recorded.replace('--', np.nan)
has_transcript.date_recorded = pd.to_datetime(has_transcript.date_recorded, format = '%Y-%m-%d')

has_transcript.upload_date = has_transcript.upload_date.replace('--', np.nan)
has_transcript.upload_date = pd.to_datetime(has_transcript.upload_date, format = '%Y-%m-%d')

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [33]:
def process_tags(tags):
    return tags.strip('[').strip(']').strip('\'').split('\', \'')

In [34]:
has_transcript['tags'] = has_transcript['tags'].apply(process_tags)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
has_transcript['tag_len'] = has_transcript['tags'].apply(len)
has_transcript['transcript_cc'] = has_transcript['transcript'].apply(len)
has_transcript['transcript_wc'] = has_transcript['transcript'].apply(lambda transcript: len(str(transcript).split()))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.or

In [36]:
has_transcript.shape

(3646, 19)

In [37]:
has_transcript.reset_index(drop = True, inplace = True)

In [38]:
import pickle

# Save to lda model
pkl_filename = "final_raw_data.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(has_transcript, file)

### Drop Ted Talks that don't have views listed or 0 views

In [27]:
has_transcript.views.value_counts()

--          39
0            8
1477907      2
1367741      2
2393761      2
1014756      1
1946761      1
657035       1
2622571      1
2035973      1
1288142      1
1048772      1
1441928      1
1753981      1
1992913      1
1308264      1
362095       1
2726173      1
3737550      1
5101478      1
1303476      1
790363       1
1605628      1
1981445      1
3013782      1
1723666      1
1617695      1
1543941      1
867110       1
11788235     1
            ..
667540       1
1394472      1
7766684      1
517391       1
4275390      1
618705       1
3169127      1
484988       1
2169686      1
2227977      1
1347395      1
1757855      1
1050151      1
346978       1
24255597     1
1872524      1
1182517      1
9612471      1
2001358      1
1028565      1
1572030      1
6632702      1
2111755      1
895723       1
3072362      1
1749826      1
2465206      1
1585904      1
958577       1
18011        1
Name: views, Length: 3598, dtype: int64

In [28]:
has_views = has_transcript[(has_transcript.views != '--')]

In [29]:
has_views.views = has_views.views.astype(int)

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [30]:
has_views = has_views[has_views.views > 0]

In [31]:
has_views.to_csv('has_views.csv')